# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ulsteinvik,62.3432,5.8487,4.62,86,99,2.24,NO,1703875878
1,1,berezovo,63.9381,65.0419,-7.04,93,100,10.06,RU,1703875879
2,2,sungai penuh,-2.0833,101.3833,18.53,98,77,1.21,ID,1703875879
3,3,touros,-5.1989,-35.4608,28.55,63,17,6.54,BR,1703875880
4,4,aykhal,66.0000,111.5000,-28.87,96,100,3.71,RU,1703875880


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:

# Configure the map plot
coordinates_df = city_data_df[["Lat","Lng","City","Humidity"]]

# Display the map
map_plot_1 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    alpha = 0.5,
    color = "City"
)
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
city_data_df_clean = city_data_df.dropna()
city_data_df_clean = city_data_df_clean[(city_data_df_clean['Max Temp']>=21) & (city_data_df_clean['Max Temp']<=27)]

# Display sample data
city_data_df_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,west island,-12.1568,96.8225,26.99,89,40,5.66,CC,1703875881
12,12,adamstown,-25.0660,-130.1015,22.58,76,88,8.14,PN,1703875883
16,16,wanning,18.8003,110.3967,21.81,92,100,3.40,CN,1703875884
17,17,kikwit,-5.0410,18.8162,23.06,92,100,0.40,CD,1703875885
22,22,iringal,11.5593,75.6110,25.57,78,91,2.51,IN,1703875886
...,...,...,...,...,...,...,...,...,...,...
568,568,pakse,15.1167,105.7833,21.54,84,97,1.43,LA,1703876056
569,569,siem reap,13.5000,104.0000,24.27,74,99,1.61,KH,1703876057
570,570,road town,18.4167,-64.6167,25.78,69,7,4.04,VG,1703876057
571,571,poxoreo,-15.8372,-54.3892,25.11,81,100,1.36,BR,1703876057


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_clean[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:,['Hotel Name']] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,west island,CC,-12.1568,96.8225,89,
12,adamstown,PN,-25.0660,-130.1015,76,
16,wanning,CN,18.8003,110.3967,92,
17,kikwit,CD,-5.0410,18.8162,92,
22,iringal,IN,11.5593,75.6110,78,
...,...,...,...,...,...,...
568,pakse,LA,15.1167,105.7833,84,
569,siem reap,KH,13.5000,104.0000,74,
570,road town,VG,18.4167,-64.6167,69,
571,poxoreo,BR,-15.8372,-54.3892,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
hotel = ''
params = {
    "filter":'filters',
    "bias":'bias',
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = row['Lng']
    Lat = row['Lat']  
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"

    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
west island - nearest hotel: Cocos Village Bungalows
adamstown - nearest hotel: No hotel found
wanning - nearest hotel: No hotel found
kikwit - nearest hotel: Grand Hôtel de Kikwit
iringal - nearest hotel: Sreemani Tourist Home
tadine - nearest hotel: Hôtel Nengoné
ati - nearest hotel: No hotel found
wamba - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
chonchi - nearest hotel: Hotel Huildín
bredasdorp - nearest hotel: Victoria Hotel
antsiranana - nearest hotel: Central
tazacorte - nearest hotel: App Leyma
namibe - nearest hotel: Hotel Chik Chik Namibe
bama - nearest hotel: No hotel found
talcahuano - nearest hotel: Club Recreativo Armada de Chile
sur - nearest hotel: Sur Hotel
pisco - nearest hotel: La Portada
bikenibeu village - nearest hotel: Otintaai Hotel
ambam - nearest hotel: Hotel L´Escapade
kone - nearest hotel: Pacifik Appartels
ishigaki - nearest hotel: アートホテル石垣島
beira - nearest hotel: Hotel Savoy
bondo - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
5,west island,CC,-12.1568,96.8225,89,Cocos Village Bungalows
12,adamstown,PN,-25.0660,-130.1015,76,No hotel found
16,wanning,CN,18.8003,110.3967,92,No hotel found
17,kikwit,CD,-5.0410,18.8162,92,Grand Hôtel de Kikwit
22,iringal,IN,11.5593,75.6110,78,Sreemani Tourist Home
...,...,...,...,...,...,...
568,pakse,LA,15.1167,105.7833,84,Residence Sisouk
569,siem reap,KH,13.5000,104.0000,74,No hotel found
570,road town,VG,18.4167,-64.6167,69,No hotel found
571,poxoreo,BR,-15.8372,-54.3892,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:

# Configure the map plot
coordinates_df = hotel_df[["Lat","Lng","City","Humidity","Hotel Name", "Country"]]

# Display the map
# Display the map
map_plot_2 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    alpha = 0.5,
    color = "City",
    hover_cols = "all"
)
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Hotel Name,Country)